In [ ]:
import rasterio as rio
import geopandas as gpd
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
city_limit=gpd.read_file("https://opendata.arcgis.com/datasets/405ec3da942d4e20869d4e1449a2be48_0.geojson")

In [ ]:
landsat=rio.open("../data/day13/landsat8_philly.tif")

In [ ]:
city_limit = city_limit.to_crs(epsg=landsat.crs.to_epsg())

In [ ]:
landsat.crs

In [ ]:
# Plot (georeferenced) with scaled RGB bands
# Scale bands to 0-255 (uint8) for display, compute image extent from raster bounds
red = landsat.read(4).astype(float)
green = landsat.read(3).astype(float)
blue = landsat.read(2).astype(float)

def scale_band(b):
    b = b.astype(float)
    b_min = np.nanmin(b)
    b_max = np.nanmax(b)
    # avoid division by zero
    if b_max - b_min == 0:
        return np.zeros(b.shape, dtype=np.uint8)
    norm = (b - b_min) / (b_max - b_min)
    return (np.clip(norm, 0, 1) * 255).astype(np.uint8)

r = scale_band(red)
g = scale_band(green)
b = scale_band(blue)

rgb = np.dstack((r, g, b))

# Get raster bounds (left, bottom, right, top) and use as extent so the GeoDataFrame overlays correctly
bounds = landsat.bounds
extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(rgb, extent=extent, origin='upper')
# city_limit should already be in the raster CRS; plot its boundary in the same coordinate space
city_limit.boundary.plot(ax=ax, edgecolor='yellow', linewidth=2)
ax.set_title("Landsat 8 Image of Philadelphia with City Limits", fontsize=16)
ax.axis('off')
plt.show()